In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn
import numpy as np
import matplotlib.pyplot as plt

# Get datasets

In [2]:
def get_features_and_outputs_from_simulations(output_name, timestamp):
    """
    Reads the simulation parameter map, fetches Cassandra to obtain the correspoding output for the specified timestamp.
    Splits the data into train/test sets with, returns c3.Dataset for each one of them.
    
    
    Inputs:
    - str output_name: name of the variable in SimulationModelOutput under investigation
    - str timestamp: shape of the timestamp to fetch the timeseries, e.g. "2017-08-19T09:00:00.000"
    
    Returns:
    - c3.Dataset X_train: train set of features
    - c3.Dataset X_test: test set of features
    - c3.Dataset y_train: train set of outputs
    - c3.Dataset y_test: test set of outputs

    """
    import pandas as pd
    from sklearn.model_selection import train_test_split
    
    # fetch simulation parameters
    parameters = c3.SimulationModelParameters.fetch().objs
    parameters = parameters.toJson()
    df = pd.DataFrame(parameters)
    simulations = pd.DataFrame(df['id'])
    X = df[df.columns[5:]]
    
    # define simple metric
    metric_name = "Average_" + output_name + "_SimulationSample" 
    metric_descr = "Calculates average of " + output_name + " for a given set of SimulationSample"
    metric_expr = "avg(avg(normalized.data." + output_name + "))"
    metric = c3.SimpleMetric(id = metric_name,
                                    name = metric_name,
                                    description = metric_descr,
                                    srcType = "SimulationSample",
                                    path = "output",  # the timeseries is in the output field of SimSam
                                    expression = metric_expr
                                  )
    
    # define metric spec
    spec = c3.EvalMetricsSpec(
                                    ids = simulations['id'],
                                    expressions = [metric_name],
                                    start = timestamp,
                                    end = timestamp,
                                    interval = "SECOND" 
                                )

    # evaluate metric, cast it to pandas
    evalMetricsResult = c3.SimulationSample.evalMetricsWithMetadata(
                                                                        spec=spec,
                                                                        overrideMetrics=[metric]
                                                                    )
    y = c3.EvalMetricsResult.toPandas(result=evalMetricsResult)
    
    
    # split into train/test sets
    datasets = train_test_split(X, y, test_size=0.1, random_state=42)
    
    # cast into c3 Datasets
    X_train = c3.Dataset.fromPython(datasets[0])
    X_test = c3.Dataset.fromPython(datasets[1])
    y_train = c3.Dataset.fromPython(datasets[2])
    y_test = c3.Dataset.fromPython(datasets[3])
    
    return X_train, X_test, y_train, y_test

In [3]:
X_train, X_test, y_train, y_test = get_features_and_outputs_from_simulations("mass_BC_acc", "2017-08-19T09:00:00.000")

500 - NotClassified - c3.service.metric.CompoundMetricEvaluator_throwError [4571.28342]
message: "c3.love.exceptions.C3RuntimeException: c3.love.exceptions.C3RuntimeException: MetricEngine error : c3.love.exceptions.C3RuntimeException: Error c3.love.exceptions.C3RuntimeException: wrapped com.datastax.driver.core.exceptions.UnauthorizedException: Unable to perform authorization of permissions: Unable to perform authorization of super-user permission: Cannot achieve consistency level QUORUM

    at com.datastax.driver.core.exceptions.UnauthorizedException.copy
    ...
    at c3.engine.database.cassandra.connector.cql.CassandraCQLClient.lambda$executeWithRetry$3 (CassandraCQLClient.java:185)
	at c3.love.exceptions.C3RuntimeException.wrapIt(C3RuntimeException.java:136)
	at c3.love.exceptions.C3RuntimeException.wrapIt(C3RuntimeException.java:85)
	at c3.love.exceptions.C3X.wrap(C3X.java:230)
	at c3.love.util.RetryMethods.now(RetryMethods.java:141)
	at c3.type.metadata.Retry.now(Retry.java:24

C3RuntimeException: 500 - NotClassified - c3.service.metric.CompoundMetricEvaluator_throwError [4571.28342]
message: "c3.love.exceptions.C3RuntimeException: c3.love.exceptions.C3RuntimeException: MetricEngine error : c3.love.exceptions.C3RuntimeException: Error c3.love.exceptions.C3RuntimeException: wrapped com.datastax.driver.core.exceptions.UnauthorizedException: Unable to perform authorization of permissions: Unable to perform authorization of super-user permission: Cannot achieve consistency level QUORUM

    at com.datastax.driver.core.exceptions.UnauthorizedException.copy
    ...
    at c3.engine.database.cassandra.connector.cql.CassandraCQLClient.lambda$executeWithRetry$3 (CassandraCQLClient.java:185)
	at c3.love.exceptions.C3RuntimeException.wrapIt(C3RuntimeException.java:136)
	at c3.love.exceptions.C3RuntimeException.wrapIt(C3RuntimeException.java:85)
	at c3.love.exceptions.C3X.wrap(C3X.java:230)
	at c3.love.util.RetryMethods.now(RetryMethods.java:141)
	at c3.type.metadata.Ret

# Train GP Regression

In [6]:
# create kernel
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=[1.0]*X_train.shape[1], nu=0.5).build().kernel

# define technique
GPReg_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
                )

# create pipe
GPReg_pipe = c3.GaussianProcessRegressionPipe(technique=GPReg_technique)

# train it
#trained_GPReg_pipe = GPReg_pipe.train(input=X_train, targetOutput=y_train)

In [26]:
# create kernel
length_scales = c3.Dataset.fromPython(np.array([1.0]*X_train.shape[1]))
GPR_kernel = c3.SklearnGPRKernelMatern(lengthScale=length_scales, nu=0.5).build().kernel

# define technique
GPReg_technique = c3.GaussianProcessRegressionTechnique(
                    randomState=42,
                    kernel = GPR_kernel
                )

# create pipe
GPReg_pipe = c3.GaussianProcessRegressionPipe(technique=GPReg_technique)

# train it
trained_GPReg_pipe = GPReg_pipe.train(input=X_train, targetOutput=y_train)

AttributeError: Error setting field "lengthScale" on instance of class "SklearnGPRKernelMatern":
Incorrect class! Expected [double] but received Dataset

In [15]:
c3.PythonSerialization.deserialize(serialized=trained_GPReg_pipe.technique.kernel.pickledKernel)

NameError: name 'trained_GPReg_pipe' is not defined

In [24]:
# check to see if something has been pickled
assert trained_GPReg_pipe.trainedModel is not None

# Predictions

In [25]:
y_test_preds = trained_GPReg_pipe.process(input=X_test)

In [26]:
y_test_preds = c3.Dataset.toPandas(dataset=y_test_preds)
y_test_preds

,0
0,2.189009e-10
1,1.120215e-10
2,1.445949e-10
3,1.332445e-10
4,1.401119e-10
5,2.673056e-10
6,9.511580e-11
7,1.093421e-10
8,7.894851e-11
9,1.684890e-10
